# Our Task: Find similar company names

Now that we have completed our Python Primer, let's try to complete a real task.

Quite often, in our data, we have entries represented as strings that refer to the same entity but have different string representations (e.g., McDonald's vs McDonalds vs McDonald‎). We want to write code that will help in the task of finding such similar entries in our data.

Our task can be broken in the following tasks:
* **Step 1**: Read the data from a file into a list of strings in memory (one string per line). For our source of company names, we can use either the list of restaurant names from the Restaurant Inspection dataset (see our prior session), and/or use a list of companies from a file available at https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/companies.txt. We need to write Python code that will read these files and return a list of company names.
* **Step 2**: We will then need to figure out how to compare two strings and find their similarity. For that, we need to write a function that takes as input two strings, and returns back their similarities (we will see below how to do that).
* **Step 3**: We will need to write a function that takes as input a company name, and returns back a list of matching company names, together with their similarity. Ideally, we would like to also sort the list based on the similarity (highest similarity metrics on top). As part of our learning process, we will use the _list comprehension_ approach to create the list. We will also use tuples as the elements of the list, so that the list contain elements such as `[("McDonalds", 0.88), ("McDonald's", 0.81),....]`.
* **Step 4**: In the final step, we will just perform the similarity computation across all companies in the dataset.


### STEP 0: Fetch the data

In [2]:
# This is a list of banks
!curl https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/companies.txt -o /home/ubuntu/data/companies.txt
    
# This is a list of NYC restaurants (also using this dataset for the regular expression notebooks)
!curl https://dl.dropboxusercontent.com/u/16006464/DwD_Winter2015/uniquenames.txt -o /home/ubuntu/data/uniquenames.txt   

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 22747  100 22747    0     0  55930      0 --:--:-- --:--:-- --:--:-- 56027
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  364k  100  364k    0     0  1753k      0 --:--:-- --:--:-- --:--:-- 1759k


In [2]:
# This is a list of banks
!wc -l /home/ubuntu/data/companies.txt
!head -5 /home/ubuntu/data/companies.txt

869 /home/ubuntu/data/companies.txt
4PROFIT VERLAG GESELLSCHAFT.M.B.H
?RXBANK OJSC
A FILM+CO. CLASSICS GMBH
A.T. KEARNEY GES.M.B.H.
AB BANK LTD


In [ ]:
# This is a list of NYC restaurants
!wc -l /home/ubuntu/data/uniquenames.txt
!head -5 /home/ubuntu/data/uniquenames.txt

### STEP 1: Read the list of names from a file and create a list of names

In [3]:
# STEP 1: Read the list of names from a file and create a list of names

f = open("/home/ubuntu/data/uniquenames.txt", "r")
content = f.read()
companies = content.split("\n")


In [4]:
# How many companies?
print len(companies)

# Let's peek at the first 10 names
print companies[:10]

20444
['#1 GARDEN CHINESE', "#1 ME. NICK'S", '#1 SABOR LATINO RESTAURANT', '$1.25 PIZZA', "''U'' LIKE CHINESE RESTAURANT", "''W'' CAFE", "'WICHCRAFT", '(LEWIS DRUG STORE) LOCANDA VINI E OLII', '(LIBRARY)  FOUR & TWENTY BLACKBIRDS', '(PUBLIC FARE) 81ST STREET AND CENTRAL PARK WEST (DELACORTE THEATRE)']


### STEP 2: Implement the similarity function

### Computing the similarity between two strings

There are many ways that we can calculate the similarity between two strings (e.g., see [1](http://en.wikipedia.org/wiki/String_metric), [2](http://en.wikipedia.org/wiki/Approximate_string_matching), [3](http://en.wikipedia.org/wiki/Plagiarism_detection), [4](https://www.cs.cmu.edu/~pradeepr/papers/ijcai03.pdf), [5](http://en.wikipedia.org/wiki/Category:String_similarity_measures), and links within). 

For our example, we will use the q-gram similarity metric. What is a q-gram? Simply a sequency of q-consecutive characters in the string. For example, the name "Panos" has the following 2-grams: "Pa", "an", "no", "os". (We can also add "#P" and "s#" if we want to capture the prefix and suffixes.) Strings that share a large number of q-grams are often similar. 

Hence, we can compute the similarity of two strings by first computing the set of q-grams for each string, and then compute the Jaccard coefficient between these sets. The [Jaccard coefficient](http://en.wikipedia.org/wiki/Jaccard_index) is simply defined as the size of the intersection divided by the size of the union of the two sets.

So, our task can be broken into two functions: One function takes a string and returns a list of the q-grams for the string, and the other takes as input two sets, and returns back their Jaccard coefficient.

In [28]:
# STEP 2: Now we implement our similarity function

# This returns a list of q-grams for a name
def getQgrams(name, q):
    # We pad the string to get the q-grams for the beginning and end    
    padded_name = "#"*(q-1) + name.upper() + "#"*(q-1)
    qgrams = []
    # We run a "sliding window" over the string to create the q-grams
    for i in range(len(name) + (q-1)):
        qgram = padded_name[i:i+q]
        qgrams.append(qgram)
    return set(qgrams)

# This function takes as input two names, computes their qgrams, and then computes
# the Jaccard coefficient (=intersection/union) of the two sets of qgrams
def computeSimilarity(name1, name2, q=3):
    set1 =  getQgrams(name1, q)
    set2 =  getQgrams(name2, q)
    intersection = set1 & set2
    union = set1 | set2
    similarity = 1.0*len(intersection) / len(union)
    return similarity

### STEP 3: Compute similarity of a company against a list of company names

We now create a function that accepts a company name and a list of companies, and computes their similarity. This part will get us to exercise our for-loops, and also illustrate how we can use lists and tuples.

**Sorting a list of tuples**:_This part is a little bit advanced for now, so I will just give the code below. (Solution taken from http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples). Here is a small example below, which we will reuse in our function:_

In [13]:
data = [("Panos",0.5), ("Peter",0.6), ("Pan", 0.4)]
data.sort(key=lambda tupl: tupl[1], reverse=True)  # sorts in place, in descending order, based on the second element of the tuple
print data

[('Peter', 0.6), ('Panos', 0.5), ('Pan', 0.4)]


In [14]:
# STEP 3: We now create a function that accepts a company name
# and a list of companies, and computes their similarity
# We have a threshold parameter (by default set to be 0.7)
# that restricts the results to only string pairs with similarity
# above the threshold
def companySimilarity(companyName, companyList, threshold = 0.7):
    result = []
    for company in companyList:
        if (companyName == company):
            continue
        similarity = computeSimilarity(companyName, company)
        if similarity>threshold:
            result.append( (company, similarity) )
            
    # Sort the list, based on the second element of the tuple
    # See  http://stackoverflow.com/questions/3121979/how-to-sort-list-tuple-of-lists-tuples
    result.sort(key=lambda tupl: tupl[1], reverse=True)
    return result

### Step 4: Perform the similarity computation across all companies in the dataset.

In [15]:
# STEP 4: We are almost done. We now just go through all the companies in the list
# and we call the companySimilarity function that computes the similar company names
# for all the companies in the list. We store the results in a dictionary, with the 
# key being the company name, and the value being a "list of tuples" with the 
# similar company names and the corresponding similarity value.
matches = {}
for c in companies:
    similarCompanies = companySimilarity(c, companies)
    if len(similarCompanies)>0:
        matches[c] = similarCompanies
        print c, "==>", similarCompanies
        print "----"



#1 SABOR LATINO RESTAURANT ==> [('SABOR LATINO RESTAURANT', 0.7666666666666667)]
----
''U'' LIKE CHINESE RESTAURANT ==> [('U LIKE CHINESE RESTAURANT', 0.7058823529411765)]
----
18 RESTAURANT ==> [('218 RESTAURANT', 0.7222222222222222)]
----
218 RESTAURANT ==> [('18 RESTAURANT', 0.7222222222222222)]
----
2647 DOUBLE DRAGON CHINESE RESTAURANT ==> [('DOUBLE DRAGON CHINESE RESTAURANT', 0.7804878048780488), ('NEW DOUBLE DRAGON CHINESE RESTAURANT', 0.75)]
----
401 LUCKY STAR RESTAURANT ==> [('LUCKY STAR RESTAURANT', 0.7142857142857143)]
----
5 STAR CHEESE STEAK AND PIZZA ==> [('5 STAR CHEESESTEAK AND PIZZA', 0.875)]
----
5 STAR CHEESESTEAK AND PIZZA ==> [('5 STAR CHEESE STEAK AND PIZZA', 0.875)]
----
85 CHINESE RESTAURANT ==> [('88 CHINESE RESTAURANT', 0.7692307692307693), ('A+ CHINESE RESTAURANT', 0.7037037037037037), ('WU CHINESE RESTAURANT', 0.7037037037037037)]
----
88 CHINESE RESTAURANT ==> [('85 CHINESE RESTAURANT', 0.7692307692307693), ('NO 8 CHINESE RESTAURANT', 0.7142857142857143), 